In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Processing data

This notebook provides an example for how `H5Data.apply()` can be used with pre-defined functions to process raw array data.

In [ ]:
from e11 import H5Data
from e11.tools import add_column_index
from e11.process import vrange, total

In [ ]:
# read file
import os 
fil = os.path.join(os.getcwd(), 'example_data', 'array_data.h5')
h5 = H5Data(fil)

## Vrange

Here, we are applying the `vrange` function to measure the vertical range of array data.

In [ ]:
rng, info = h5.apply(vrange, 'OSC_0', info=True)
rng.head()

In [ ]:
rng.describe()

In [ ]:
# information about the processing
pd.DataFrame([info])

## Total 

Simularly, we can apply the `total` function to measure the sum of array data inside a given window.

In [ ]:
tot = h5.apply(total, 'OSC_0', window=(100, 300))
tot.head()

In [ ]:
# plot
tot.reset_index().plot(subplots=True)

#output
plt.show()